## Calculate the averge loss and standard deviation of the loss

In [ ]:
import os
import re
from collections import defaultdict
from statistics import mean, stdev

def parse_final_losses_from_file(filepath):
    with open(filepath, 'r') as file:
        content = file.read()

    # Split content by blocks between delimiter lines
    blocks = content.split("=" * 80)

    model_losses = defaultdict(list)

    for block in blocks:
        model_match = re.search(r'([^\s]+)\s*->\s*([^\n]+)', block)
        loss_match = re.search(r'Final Loss:\s*([0-9.]+)', block)

        if model_match and loss_match:
            teacher_model = model_match.group(1).strip()
            student_model = model_match.group(2).strip()
            loss = float(loss_match.group(1))

            # Aggregate by teacher -> student combination
            pair_key = f"{teacher_model} -> {student_model}"
            model_losses[pair_key].append(loss)
    return model_losses

def aggregate_losses(folder_path):
    aggregated_losses = defaultdict(list)

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                file_losses = parse_final_losses_from_file(file_path)
                for model, losses in file_losses.items():
                    aggregated_losses[model].extend(losses)
                    
    summary = {}
    for model_pair, losses in aggregated_losses.items():
        avg = mean(losses)
        std = stdev(losses) if len(losses) > 1 else 0.0
        summary[model_pair] = (avg, std)

    return summary


In [ ]:
folder = "../experiment_output/Experiment1"
results = aggregate_losses(folder)

print("Average Final Loss ± Std per Teacher -> Student Model:")
for model_pair, (avg_loss, std_loss) in sorted(results.items()):
    print(f"{model_pair}: {avg_loss:.4f} ± {std_loss:.4f}")


Average Final Loss ± Std per Teacher -> Student Model:
nonoverlappingCNN_relu -> nonoverlappingCNN_relu: 2.4908 ± 2.2822
nonoverlappingCNN_relu -> nonoverlappingCNN_sigmoid: 3.5135 ± 0.1102
nonoverlappingCNN_relu -> nonoverlappingCNN_tanh: 3.6827 ± 0.1164
nonoverlappingCNN_sigmoid -> nonoverlappingCNN_relu: 0.0896 ± 0.0011
nonoverlappingCNN_sigmoid -> nonoverlappingCNN_sigmoid: 0.0021 ± 0.0011
nonoverlappingCNN_sigmoid -> nonoverlappingCNN_tanh: 0.2523 ± 0.0002
nonoverlappingCNN_tanh -> nonoverlappingCNN_relu: 0.3110 ± 0.0306
nonoverlappingCNN_tanh -> nonoverlappingCNN_sigmoid: 0.2668 ± 0.0938
nonoverlappingCNN_tanh -> nonoverlappingCNN_tanh: 0.0001 ± 0.0000
